# 06 - 3D Geographic Data Visualization

## Learning Objectives
- Master ECharts GL 3D geographic visualization technology
- Create interactive 3D maps and geographic data displays
- Implement real-time geographic data analysis and visualization

## Difficulty Level: ⭐⭐⭐⭐☆

## Core Concepts

Kotlin Geographic Data → 3D Coordinate Transformation → ECharts GL Rendering → Interactive Maps

In [1]:
// 🔧 Local debug version
// Note: To use local version, first run ./gradlew publishToMavenLocal to build local version
USE {
    repositories {
        // mavenLocal()
        mavenCentral()
    }
    dependencies {
        implementation("dev.yidafu.jupyter:jupyter-js:0.8.0")
    }
}


In [2]:
// Global city geographic data
val globalCities = listOf(
    mapOf(
        "name" to "Beijing",
        "country" to "China",
        "lat" to 39.9042,
        "lng" to 116.4074,
        "population" to 21540000,
        "gdp" to 603300000000L
    ),
    mapOf(
        "name" to "Shanghai",
        "country" to "China",
        "lat" to 31.2304,
        "lng" to 121.4737,
        "population" to 24280000,
        "gdp" to 563000000000L
    ),
    mapOf(
        "name" to "Shenzhen",
        "country" to "China",
        "lat" to 22.5431,
        "lng" to 114.0579,
        "population" to 12590000,
        "gdp" to 423000000000L
    ),
    mapOf(
        "name" to "Guangzhou",
        "country" to "China",
        "lat" to 23.1291,
        "lng" to 113.2644,
        "population" to 15300000,
        "gdp" to 402000000000L
    ),
    mapOf(
        "name" to "Hangzhou",
        "country" to "China",
        "lat" to 30.2741,
        "lng" to 120.1551,
        "population" to 10360000,
        "gdp" to 235000000000L
    ),
    mapOf(
        "name" to "Chengdu",
        "country" to "China",
        "lat" to 30.5728,
        "lng" to 104.0668,
        "population" to 20940000,
        "gdp" to 277000000000L
    ),
    mapOf(
        "name" to "Xi'an",
        "country" to "China",
        "lat" to 34.3416,
        "lng" to 108.9398,
        "population" to 12950000,
        "gdp" to 140000000000L
    ),
    mapOf(
        "name" to "Wuhan",
        "country" to "China",
        "lat" to 30.5928,
        "lng" to 114.3055,
        "population" to 11210000,
        "gdp" to 238000000000L
    )
)

// Inter-city connection data (simulating flights, trade routes, etc.)
val cityConnections = listOf(
    mapOf("from" to "Beijing", "to" to "Shanghai", "value" to 2800000),
    mapOf("from" to "Beijing", "to" to "Shenzhen", "value" to 2200000),
    mapOf("from" to "Beijing", "to" to "Guangzhou", "value" to 1900000),
    mapOf("from" to "Shanghai", "to" to "Shenzhen", "value" to 2400000),
    mapOf("from" to "Shanghai", "to" to "Hangzhou", "value" to 1600000),
    mapOf("from" to "Guangzhou", "to" to "Shenzhen", "value" to 3200000),
    mapOf("from" to "Chengdu", "to" to "Shanghai", "value" to 1400000),
    mapOf("from" to "Xi'an", "to" to "Beijing", "value" to 1100000)
)

// Climate data
val climateData = listOf(
    mapOf("city" to "Beijing", "temperature" to 12.5, "humidity" to 55, "rainfall" to 585),
    mapOf("city" to "Shanghai", "temperature" to 16.7, "humidity" to 70, "rainfall" to 1166),
    mapOf("city" to "Shenzhen", "temperature" to 22.4, "humidity" to 75, "rainfall" to 1935),
    mapOf("city" to "Guangzhou", "temperature" to 21.8, "humidity" to 77, "rainfall" to 1736),
    mapOf("city" to "Hangzhou", "temperature" to 16.4, "humidity" to 68, "rainfall" to 1454),
    mapOf("city" to "Chengdu", "temperature" to 16.0, "humidity" to 82, "rainfall" to 976),
    mapOf("city" to "Xi'an", "temperature" to 13.9, "humidity" to 63, "rainfall" to 553),
    mapOf("city" to "Wuhan", "temperature" to 16.6, "humidity" to 78, "rainfall" to 1269)
)

println("🌍 Geographic data preparation complete")
println("Number of cities: ${globalCities.size}")
println("Number of connections: ${cityConnections.size}")
println("Climate data: ${climateData.size} cities")

🌍 Geographic data preparation complete
Number of cities: 8
Number of connections: 8
Climate data: 8 cities


## 🌐 3D Global City Distribution

In [3]:
%js

import { globalCities } from '@jupyter';
import * as echarts from 'echarts';

const container = getContainer();
container.innerHTML = `
    <div style="font-family: 'Segoe UI', sans-serif; max-width: 1200px; margin: 0 auto; padding: 20px;">
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 12px; margin-bottom: 20px; text-align: center;">
            <h1 style="margin: 0; font-size: 28px;">🌐 3D Global City Distribution</h1>
            <p style="margin: 8px 0 0 0; opacity: 0.9;">Interactive 3D visualization of major cities - Population & GDP Analysis</p>
        </div>
        <div id="city-globe" style="width: 100%; height: 700px; border-radius: 12px; box-shadow: 0 10px 30px rgba(0,0,0,0.2);"></div>
        <div id="city-stats" style="margin-top: 20px; display: grid; grid-template-columns: repeat(4, 1fr); gap: 15px;"></div>
    </div>
`;

// Initialize ECharts
const chart = echarts.init(document.getElementById('city-globe'));

// Prepare scatter data
const scatterData = globalCities.map(city => ({
    name: city.name,
    value: [city.lng, city.lat, city.population / 1000000],
    itemStyle: {
        color: city.population > 15000000 ? '#ff6b6b' : 
               city.population > 12000000 ? '#ffd93d' : '#6bcf7f'
    },
    city: city
}));

const option = {
    backgroundColor: '#000',
    globe: {
        baseTexture: 'https://cdn.jsdelivr.net/gh/apache/echarts-website@asf-site/examples/data/asset/world.topo.bathy.200401.jpg',
        heightTexture: 'https://cdn.jsdelivr.net/gh/apache/echarts-website@asf-site/examples/data/asset/bathymetry_bw_composite_4k.jpg',
        displacementScale: 0.04,
        shading: 'realistic',
        environment: 'https://cdn.jsdelivr.net/gh/apache/echarts-website@asf-site/examples/data-gl/asset/starfield.jpg',
        realisticMaterial: {
            roughness: 0.9,
            metalness: 0
        },
        postEffect: {
            enable: true,
            bloom: {
                enable: true,
                intensity: 0.1
            }
        },
        light: {
            main: {
                intensity: 2,
                shadow: true
            },
            ambient: {
                intensity: 0.5
            }
        },
        viewControl: {
            autoRotate: true,
            autoRotateSpeed: 5,
            distance: 200,
            minDistance: 150,
            maxDistance: 400
        }
    },
    series: [{
        type: 'scatter3D',
        coordinateSystem: 'globe',
        data: scatterData,
        symbolSize: (params) => {
            return Math.sqrt(params[2]) * 3;
        },
        itemStyle: {
            opacity: 0.9
        },
        label: {
            show: true,
            position: 'top',
            formatter: '{b}',
            textStyle: {
                color: '#fff',
                fontSize: 12,
                borderWidth: 1,
                borderColor: '#000',
                backgroundColor: 'rgba(0,0,0,0.5)',
                padding: [4, 8],
                borderRadius: 4
            }
        },
        emphasis: {
            itemStyle: {
                color: '#fff'
            },
            label: {
                show: true,
                fontSize: 14
            }
        }
    }]
};

chart.setOption(option);

// Display statistics
const totalPop = globalCities.reduce((sum, city) => sum + city.population, 0);
const totalGDP = globalCities.reduce((sum, city) => sum + Number(city.gdp), 0);
const avgPop = totalPop / globalCities.length;
const maxCity = globalCities.reduce((max, city) => 
    city.population > max.population ? city : max, globalCities[0]);

document.getElementById('city-stats').innerHTML = `
    <div style="padding: 15px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 8px; text-align: center; color: white;">
        <div style="font-size: 14px; opacity: 0.9;">Total Cities</div>
        <div style="font-size: 24px; font-weight: bold; margin-top: 5px;">${globalCities.length}</div>
    </div>
    <div style="padding: 15px; background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); border-radius: 8px; text-align: center; color: white;">
        <div style="font-size: 14px; opacity: 0.9;">Total Population</div>
        <div style="font-size: 24px; font-weight: bold; margin-top: 5px;">${(totalPop / 1000000).toFixed(1)}M</div>
    </div>
    <div style="padding: 15px; background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); border-radius: 8px; text-align: center; color: white;">
        <div style="font-size: 14px; opacity: 0.9;">Total GDP</div>
        <div style="font-size: 24px; font-weight: bold; margin-top: 5px;">$${(totalGDP / 1000000000000).toFixed(2)}T</div>
    </div>
    <div style="padding: 15px; background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%); border-radius: 8px; text-align: center; color: white;">
        <div style="font-size: 14px; opacity: 0.9;">Largest City</div>
        <div style="font-size: 24px; font-weight: bold; margin-top: 5px;">${maxCity.name}</div>
    </div>
`;

// Tooltip
chart.on('mouseover', (params) => {
    if (params.componentType === 'series') {
        const city = params.data.city;
        console.log(`${city.name}: Pop ${(city.population/1000000).toFixed(1)}M, GDP $${(city.gdp/1000000000).toFixed(1)}B`);
    }
});

// Resize handler
window.addEventListener('resize', () => chart.resize());

## ✈️ 3D Flight Network Visualization

In [4]:
%js

import { globalCities, cityConnections } from '@jupyter';
import * as echarts from 'echarts';
import 'echarts-gl';

const container = getContainer();
container.innerHTML = `
    <div style="font-family: 'Segoe UI', sans-serif; max-width: 1200px; margin: 0 auto; padding: 20px;">
        <div style="background: linear-gradient(135deg, #fa709a 0%, #fee140 100%); color: white; padding: 25px; border-radius: 12px; margin-bottom: 20px; text-align: center;">
            <h1 style="margin: 0; font-size: 28px;">✈️ 3D Flight Network Visualization</h1>
            <p style="margin: 8px 0 0 0; opacity: 0.9;">Interactive city connection network with flow animation</p>
        </div>
        <div id="flight-network" style="width: 100%; height: 700px; border-radius: 12px; box-shadow: 0 10px 30px rgba(0,0,0,0.2);"></div>
        <div id="network-stats" style="margin-top: 20px; display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px;"></div>
    </div>
`;

const chart = echarts.init(document.getElementById('flight-network'));

// Create city lookup map
const cityMap = {};
globalCities.forEach(city => {
    cityMap[city.name] = city;
});

// Prepare geo coordinates for map
const geoCoords = {};
globalCities.forEach(city => {
    geoCoords[city.name] = [city.lng, city.lat];
});

// Prepare connection lines data
const linesData = cityConnections.map(conn => ({
    coords: [
        geoCoords[conn.from],
        geoCoords[conn.to]
    ],
    value: conn.value,
    lineStyle: {
        color: new echarts.graphic.LinearGradient(0, 0, 1, 0, [{
            offset: 0,
            color: '#ff6b6b'
        }, {
            offset: 1,
            color: '#4ecdc4'
        }]),
        width: Math.sqrt(conn.value) / 500,
        curveness: 0.3,
        opacity: 0.6
    }
}));

// Prepare scatter data for cities
const scatterData = globalCities.map(city => ({
    name: city.name,
    value: [city.lng, city.lat, city.population / 1000000],
    symbolSize: Math.sqrt(city.population) / 1000,
    itemStyle: {
        color: '#ffd93d'
    }
}));

const option = {
    backgroundColor: '#0a0a1a',
    geo3D: {
        map: 'china',
        roam: true,
        boxHeight: 5,
        regionHeight: 3,
        shading: 'realistic',
        realisticMaterial: {
            detailTexture: '#fff',
            textureTiling: 1,
            roughness: 0.5,
            metalness: 0
        },
        postEffect: {
            enable: true,
            bloom: {
                enable: true,
                intensity: 0.1
            }
        },
        light: {
            main: {
                intensity: 1.5,
                shadow: true,
                shadowQuality: 'high'
            },
            ambient: {
                intensity: 0.3
            }
        },
        viewControl: {
            distance: 100,
            alpha: 30,
            beta: 30,
            autoRotate: true,
            autoRotateSpeed: 10
        },
        itemStyle: {
            color: '#1a1a3e',
            opacity: 0.9,
            borderWidth: 0.8,
            borderColor: '#4ecdc4'
        },
        emphasis: {
            itemStyle: {
                color: '#2a2a5e'
            }
        }
    },
    series: [
        {
            type: 'lines3D',
            coordinateSystem: 'geo3D',
            data: linesData,
            effect: {
                show: true,
                period: 4,
                trailLength: 0.7,
                color: '#fff',
                symbolSize: 3
            },
            blendMode: 'lighter'
        },
        {
            type: 'scatter3D',
            coordinateSystem: 'geo3D',
            data: scatterData,
            symbol: 'circle',
            itemStyle: {
                color: '#ffd93d',
                opacity: 1
            },
            label: {
                show: true,
                position: 'top',
                formatter: '{b}',
                textStyle: {
                    color: '#fff',
                    fontSize: 11,
                    borderWidth: 1,
                    borderColor: 'rgba(0,0,0,0.8)',
                    backgroundColor: 'rgba(0,0,0,0.5)',
                    padding: [3, 6],
                    borderRadius: 3
                }
            },
            emphasis: {
                itemStyle: {
                    color: '#ff6b6b'
                }
            }
        }
    ]
};

// Load China map
fetch('https://cdn.jsdelivr.net/gh/apache/echarts-website@asf-site/examples/data/asset/geo/china.json')
    .then(response => response.json())
    .then(chinaJson => {
        echarts.registerMap('china', chinaJson);
        chart.setOption(option);
    });

// Display statistics
const totalConnections = cityConnections.length;
const totalTraffic = cityConnections.reduce((sum, conn) => sum + conn.value, 0);
const avgTraffic = totalTraffic / totalConnections;

document.getElementById('network-stats').innerHTML = `
    <div style="padding: 15px; background: linear-gradient(135deg, #fa709a 0%, #fee140 100%); border-radius: 8px; text-align: center; color: white;">
        <div style="font-size: 14px; opacity: 0.9;">Total Routes</div>
        <div style="font-size: 28px; font-weight: bold; margin-top: 5px;">${totalConnections}</div>
    </div>
    <div style="padding: 15px; background: linear-gradient(135deg, #30cfd0 0%, #330867 100%); border-radius: 8px; text-align: center; color: white;">
        <div style="font-size: 14px; opacity: 0.9;">Total Traffic</div>
        <div style="font-size: 28px; font-weight: bold; margin-top: 5px;">${(totalTraffic / 1000000).toFixed(1)}M</div>
    </div>
    <div style="padding: 15px; background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%); border-radius: 8px; text-align: center; color: #333;">
        <div style="font-size: 14px; opacity: 0.9;">Avg per Route</div>
        <div style="font-size: 28px; font-weight: bold; margin-top: 5px;">${(avgTraffic / 1000000).toFixed(2)}M</div>
    </div>
`;

window.addEventListener('resize', () => chart.resize());

## 🌡️ Climate dataradar chart

In [5]:
%js

import { climateData, globalCities } from '@jupyter';
import * as echarts from 'echarts';

const container = getContainer();
container.innerHTML = `
    <div style="font-family: 'Segoe UI', sans-serif; max-width: 1200px; margin: 0 auto; padding: 20px;">
        <div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); color: white; padding: 25px; border-radius: 12px; margin-bottom: 20px; text-align: center;">
            <h1 style="margin: 0; font-size: 28px;">🌡️ Climate Data Radar Chart</h1>
            <p style="margin: 8px 0 0 0; opacity: 0.9;">Multi-dimensional climate analysis - Temperature, Humidity, Rainfall</p>
        </div>
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
            <div id="climate-radar" style="width: 100%; height: 500px; border-radius: 12px; box-shadow: 0 10px 30px rgba(0,0,0,0.1); background: white;"></div>
            <div id="climate-bars" style="width: 100%; height: 500px; border-radius: 12px; box-shadow: 0 10px 30px rgba(0,0,0,0.1); background: white;"></div>
        </div>
        <div id="climate-summary" style="margin-top: 20px; padding: 20px; background: #f8f9fa; border-radius: 8px;"></div>
    </div>
`;

// Radar Chart
const radarChart = echarts.init(document.getElementById('climate-radar'));

const radarOption = {
    title: {
        text: 'City Climate Comparison',
        left: 'center',
        top: 20
    },
    tooltip: {
        trigger: 'item'
    },
    legend: {
        type: 'scroll',
        bottom: 10,
        data: climateData.map(d => d.city)
    },
    radar: {
        indicator: [
            { name: 'Temperature (°C)', max: 25 },
            { name: 'Humidity (%)', max: 100 },
            { name: 'Rainfall (mm)', max: 2000 }
        ],
        splitNumber: 4,
        shape: 'polygon',
        axisName: {
            color: '#333',
            fontSize: 12
        },
        splitArea: {
            areaStyle: {
                color: ['rgba(79, 172, 254, 0.1)', 'rgba(79, 172, 254, 0.2)']
            }
        },
        splitLine: {
            lineStyle: {
                color: 'rgba(79, 172, 254, 0.5)'
            }
        }
    },
    series: [{
        type: 'radar',
        data: climateData.map((climate, idx) => ({
            name: climate.city,
            value: [climate.temperature, climate.humidity, climate.rainfall],
            itemStyle: {
                color: [
                    '#ff6b6b', '#4ecdc4', '#ffd93d', '#95e1d3',
                    '#f38181', '#aa96da', '#fcbad3', '#a8e6cf'
                ][idx % 8]
            },
            areaStyle: {
                opacity: 0.3
            }
        }))
    }]
};

radarChart.setOption(radarOption);

// Bar Chart for Temperature
const barChart = echarts.init(document.getElementById('climate-bars'));
const sortedByTemp = [...climateData].sort((a, b) => b.temperature - a.temperature);

const barOption = {
    title: {
        text: 'Temperature Ranking',
        left: 'center',
        top: 20
    },
    tooltip: {
        trigger: 'axis',
        axisPointer: {
            type: 'shadow'
        },
        formatter: (params) => {
            const data = params[0];
            const climate = climateData.find(c => c.city === data.name);
            return `${data.name}<br/>
                Temperature: ${climate.temperature}°C<br/>
                Humidity: ${climate.humidity}%<br/>
                Rainfall: ${climate.rainfall}mm`;
        }
    },
    grid: {
        left: '3%',
        right: '4%',
        bottom: '3%',
        top: '15%',
        containLabel: true
    },
    xAxis: {
        type: 'category',
        data: sortedByTemp.map(d => d.city),
        axisLabel: {
            interval: 0,
            rotate: 30
        }
    },
    yAxis: {
        type: 'value',
        name: 'Temperature (°C)'
    },
    series: [{
        type: 'bar',
        data: sortedByTemp.map((d, idx) => ({
            value: d.temperature,
            itemStyle: {
                color: new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                    offset: 0,
                    color: '#ff6b6b'
                }, {
                    offset: 1,
                    color: '#4ecdc4'
                }])
            }
        })),
        barWidth: '50%',
        label: {
            show: true,
            position: 'top',
            formatter: '{c}°C'
        }
    }]
};

barChart.setOption(barOption);

// Climate summary
const avgTemp = climateData.reduce((sum, d) => sum + d.temperature, 0) / climateData.length;
const avgHumidity = climateData.reduce((sum, d) => sum + d.humidity, 0) / climateData.length;
const avgRainfall = climateData.reduce((sum, d) => sum + d.rainfall, 0) / climateData.length;
const hottestCity = climateData.reduce((max, d) => d.temperature > max.temperature ? d : max);
const wettest = climateData.reduce((max, d) => d.rainfall > max.rainfall ? d : max);

document.getElementById('climate-summary').innerHTML = `
    <div style="display: grid; grid-template-columns: repeat(5, 1fr); gap: 15px;">
        <div style="text-align: center;">
            <div style="font-size: 14px; color: #666;">Avg Temperature</div>
            <div style="font-size: 24px; font-weight: bold; color: #ff6b6b; margin-top: 5px;">${avgTemp.toFixed(1)}°C</div>
        </div>
        <div style="text-align: center;">
            <div style="font-size: 14px; color: #666;">Avg Humidity</div>
            <div style="font-size: 24px; font-weight: bold; color: #4ecdc4; margin-top: 5px;">${avgHumidity.toFixed(1)}%</div>
        </div>
        <div style="text-align: center;">
            <div style="font-size: 14px; color: #666;">Avg Rainfall</div>
            <div style="font-size: 24px; font-weight: bold; color: #ffd93d; margin-top: 5px;">${avgRainfall.toFixed(0)}mm</div>
        </div>
        <div style="text-align: center;">
            <div style="font-size: 14px; color: #666;">Hottest City</div>
            <div style="font-size: 20px; font-weight: bold; color: #f38181; margin-top: 5px;">${hottestCity.city}</div>
        </div>
        <div style="text-align: center;">
            <div style="font-size: 14px; color: #666;">Wettest City</div>
            <div style="font-size: 20px; font-weight: bold; color: #95e1d3; margin-top: 5px;">${wettest.city}</div>
        </div>
    </div>
`;

window.addEventListener('resize', () => {
    radarChart.resize();
    barChart.resize();
});

## 🌊 3D Ocean Temperature Currents

In [6]:
%js

import * as echarts from 'echarts';
import 'echarts-gl';

const container = getContainer();
container.innerHTML = `
    <div style="font-family: 'Segoe UI', sans-serif; max-width: 1200px; margin: 0 auto; padding: 20px;">
        <div style="background: linear-gradient(135deg, #0093E9 0%, #80D0C7 100%); color: white; padding: 25px; border-radius: 12px; margin-bottom: 20px; text-align: center;">
            <h1 style="margin: 0; font-size: 28px;">🌊 3D Ocean Temperature Currents</h1>
            <p style="margin: 8px 0 0 0; opacity: 0.9;">Simulated sea surface temperature distribution with 3D visualization</p>
        </div>
        <div id="ocean-temp" style="width: 100%; height: 700px; border-radius: 12px; box-shadow: 0 10px 30px rgba(0,0,0,0.2);"></div>
        <div style="margin-top: 15px; padding: 15px; background: linear-gradient(90deg, #1e3c72 0%, #2a5298 50%, #7e22ce 100%); border-radius: 8px;">
            <div style="display: flex; align-items: center; justify-content: center; color: white;">
                <span style="margin-right: 20px;">Temperature Scale:</span>
                <div style="display: flex; gap: 10px; align-items: center;">
                    <div style="display: flex; align-items: center;">
                        <div style="width: 30px; height: 20px; background: #313695; margin-right: 5px;"></div>
                        <span>Cold (0-10°C)</span>
                    </div>
                    <div style="display: flex; align-items: center;">
                        <div style="width: 30px; height: 20px; background: #4575b4; margin-right: 5px;"></div>
                        <span>Cool (10-15°C)</span>
                    </div>
                    <div style="display: flex; align-items: center;">
                        <div style="width: 30px; height: 20px; background: #abd9e9; margin-right: 5px;"></div>
                        <span>Mild (15-20°C)</span>
                    </div>
                    <div style="display: flex; align-items: center;">
                        <div style="width: 30px; height: 20px; background: #fee090; margin-right: 5px;"></div>
                        <span>Warm (20-25°C)</span>
                    </div>
                    <div style="display: flex; align-items: center;">
                        <div style="width: 30px; height: 20px; background: #f46d43; margin-right: 5px;"></div>
                        <span>Hot (25-30°C)</span>
                    </div>
                    <div style="display: flex; align-items: center;">
                        <div style="width: 30px; height: 20px; background: #a50026; margin-right: 5px;"></div>
                        <span>Very Hot (30°C+)</span>
                    </div>
                </div>
            </div>
        </div>
    </div>
`;

const chart = echarts.init(document.getElementById('ocean-temp'));

// Generate ocean temperature data
const oceanData = [];
const gridX = 80;
const gridY = 60;

for (let x = 0; x < gridX; x++) {
    for (let y = 0; y < gridY; y++) {
        const lat = 70 - (y / gridY) * 140; // -70 to 70
        const lng = -180 + (x / gridX) * 360; // -180 to 180
        
        // Simulate temperature based on latitude and some noise
        const latitudeFactor = Math.abs(lat);
        const baseTemp = 30 - (latitudeFactor / 90) * 25; // Warmer at equator
        
        // Add some variation using sine waves
        const variation = Math.sin(lng / 20) * 3 + Math.cos(lat / 15) * 2;
        const noise = (Math.random() - 0.5) * 2;
        
        const temperature = baseTemp + variation + noise;
        
        oceanData.push([lng, lat, temperature]);
    }
}

const option = {
    backgroundColor: '#000',
    globe: {
        baseTexture: 'https://cdn.jsdelivr.net/gh/apache/echarts-website@asf-site/examples/data/asset/world.topo.bathy.200401.jpg',
        heightTexture: 'https://cdn.jsdelivr.net/gh/apache/echarts-website@asf-site/examples/data/asset/bathymetry_bw_composite_4k.jpg',
        displacementScale: 0.1,
        shading: 'realistic',
        environment: 'https://cdn.jsdelivr.net/gh/apache/echarts-website@asf-site/examples/data-gl/asset/starfield.jpg',
        realisticMaterial: {
            roughness: 0.2,
            metalness: 0
        },
        postEffect: {
            enable: true,
            bloom: {
                enable: true,
                intensity: 0.3
            },
            SSAO: {
                enable: true,
                radius: 2
            }
        },
        light: {
            main: {
                intensity: 3,
                shadow: false
            },
            ambient: {
                intensity: 0.5
            }
        },
        viewControl: {
            autoRotate: true,
            autoRotateSpeed: 8,
            distance: 180,
            minDistance: 120,
            maxDistance: 350,
            alpha: 25,
            beta: 0
        },
        layers: [{
            type: 'blend',
            blendTo: 'emission',
            texture: 'https://cdn.jsdelivr.net/gh/apache/echarts-website@asf-site/examples/data/asset/night.jpg',
            intensity: 0.3
        }]
    },
    visualMap: {
        show: true,
        min: 0,
        max: 35,
        calculable: true,
        realtime: true,
        inRange: {
            color: [
                '#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8',
                '#ffffbf', '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026'
            ]
        },
        textStyle: {
            color: '#fff'
        },
        bottom: 50,
        left: 50,
        text: ['High', 'Low'],
        dimension: 2,
        formatter: (value) => `${value.toFixed(1)}°C`
    },
    series: [{
        type: 'scatter3D',
        coordinateSystem: 'globe',
        data: oceanData,
        symbolSize: 1.5,
        itemStyle: {
            opacity: 0.9
        },
        blendMode: 'lighter',
        silent: true
    }]
};

chart.setOption(option);

window.addEventListener('resize', () => chart.resize());

## 📊 Geographic Data Statistics Panel

In [7]:
%js

import { globalCities, climateData } from '@jupyter';
import * as echarts from 'echarts';

const container = getContainer();
container.innerHTML = `
    <div style="font-family: 'Segoe UI', sans-serif; max-width: 1200px; margin: 0 auto; padding: 20px;">
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 12px; margin-bottom: 20px; text-align: center;">
            <h1 style="margin: 0; font-size: 28px;">📊 Geographic Data Statistics Panel</h1>
            <p style="margin: 8px 0 0 0; opacity: 0.9;">Comprehensive analysis dashboard</p>
        </div>
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin-bottom: 20px;">
            <div id="gdp-chart" style="width: 100%; height: 400px; border-radius: 12px; box-shadow: 0 5px 15px rgba(0,0,0,0.1); background: white;"></div>
            <div id="population-chart" style="width: 100%; height: 400px; border-radius: 12px; box-shadow: 0 5px 15px rgba(0,0,0,0.1); background: white;"></div>
        </div>
        <div id="combined-chart" style="width: 100%; height: 450px; border-radius: 12px; box-shadow: 0 5px 15px rgba(0,0,0,0.1); background: white;"></div>
    </div>
`;

// GDP Pie Chart
const gdpChart = echarts.init(document.getElementById('gdp-chart'));
const sortedByGDP = [...globalCities].sort((a, b) => Number(b.gdp) - Number(a.gdp));

gdpChart.setOption({
    title: {
        text: 'GDP Distribution',
        left: 'center',
        top: 20,
        textStyle: {
            fontSize: 18,
            fontWeight: 'bold'
        }
    },
    tooltip: {
        trigger: 'item',
        formatter: (params) => {
            return `${params.name}<br/>GDP: $${(params.value / 1000000000).toFixed(1)}B<br/>Percentage: ${params.percent}%`;
        }
    },
    legend: {
        orient: 'vertical',
        right: 10,
        top: 'middle',
        type: 'scroll'
    },
    series: [{
        type: 'pie',
        radius: ['40%', '70%'],
        center: ['40%', '55%'],
        data: sortedByGDP.map(city => ({
            name: city.name,
            value: Number(city.gdp)
        })),
        emphasis: {
            itemStyle: {
                shadowBlur: 10,
                shadowOffsetX: 0,
                shadowColor: 'rgba(0, 0, 0, 0.5)'
            }
        },
        label: {
            formatter: '{b}: {d}%'
        }
    }]
});

// Population Bar Chart
const popChart = echarts.init(document.getElementById('population-chart'));
const sortedByPop = [...globalCities].sort((a, b) => b.population - a.population);

popChart.setOption({
    title: {
        text: 'Population Ranking',
        left: 'center',
        top: 20,
        textStyle: {
            fontSize: 18,
            fontWeight: 'bold'
        }
    },
    tooltip: {
        trigger: 'axis',
        axisPointer: {
            type: 'shadow'
        },
        formatter: (params) => {
            return `${params[0].name}<br/>Population: ${(params[0].value / 1000000).toFixed(2)}M`;
        }
    },
    grid: {
        left: '5%',
        right: '5%',
        bottom: '5%',
        top: '20%',
        containLabel: true
    },
    xAxis: {
        type: 'category',
        data: sortedByPop.map(city => city.name),
        axisLabel: {
            interval: 0,
            rotate: 30
        }
    },
    yAxis: {
        type: 'value',
        name: 'Population (M)',
        axisLabel: {
            formatter: (value) => (value / 1000000).toFixed(0)
        }
    },
    series: [{
        type: 'bar',
        data: sortedByPop.map(city => ({
            value: city.population,
            itemStyle: {
                color: new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                    offset: 0,
                    color: '#667eea'
                }, {
                    offset: 1,
                    color: '#764ba2'
                }])
            }
        })),
        barWidth: '60%',
        label: {
            show: true,
            position: 'top',
            formatter: (params) => (params.value / 1000000).toFixed(1) + 'M',
            fontSize: 10
        }
    }]
});

// Combined Scatter Chart
const combinedChart = echarts.init(document.getElementById('combined-chart'));

combinedChart.setOption({
    title: {
        text: 'GDP vs Population Correlation',
        left: 'center',
        top: 20,
        textStyle: {
            fontSize: 18,
            fontWeight: 'bold'
        }
    },
    tooltip: {
        trigger: 'item',
        formatter: (params) => {
            const city = params.data.city;
            return `${city.name}<br/>
                Population: ${(city.population / 1000000).toFixed(2)}M<br/>
                GDP: $${(Number(city.gdp) / 1000000000).toFixed(1)}B<br/>
                GDP per capita: $${(Number(city.gdp) / city.population).toFixed(0)}`;
        }
    },
    grid: {
        left: '10%',
        right: '10%',
        bottom: '10%',
        top: '20%',
        containLabel: true
    },
    xAxis: {
        type: 'value',
        name: 'Population (Million)',
        nameLocation: 'middle',
        nameGap: 30,
        axisLabel: {
            formatter: (value) => (value / 1000000).toFixed(0)
        }
    },
    yAxis: {
        type: 'value',
        name: 'GDP (Billion USD)',
        nameLocation: 'middle',
        nameGap: 50,
        axisLabel: {
            formatter: (value) => (value / 1000000000).toFixed(0)
        }
    },
    series: [{
        type: 'scatter',
        data: globalCities.map(city => ({
            value: [city.population, Number(city.gdp)],
            symbolSize: Math.sqrt(city.population) / 100,
            city: city,
            itemStyle: {
                color: city.population > 15000000 ? '#ff6b6b' : 
                       city.population > 12000000 ? '#ffd93d' : '#6bcf7f'
            }
        })),
        emphasis: {
            itemStyle: {
                shadowBlur: 10,
                shadowColor: 'rgba(0, 0, 0, 0.5)'
            }
        },
        label: {
            show: true,
            position: 'top',
            formatter: (params) => params.data.city.name,
            fontSize: 10
        }
    }]
});

window.addEventListener('resize', () => {
    gdpChart.resize();
    popChart.resize();
    combinedChart.resize();
});

## ✅ Summary

### 🎯 Skills Mastered:
- ✅ **3D Geographic Visualization** - ECharts GL map rendering and interaction
- ✅ **Geographic Data Processing** - Coordinate transformation and data mapping
- ✅ **Multi-dimensional display** - population、economic、Climate dataintegration
- ✅ **Real-time Animation Effects** - Auto-rotation, flowing animations
- ✅ **Interactive Exploration** - Zoom, rotate, hover tooltips

### 🌟 Real-World Application Scenarios:
- 🌍 Global business distribution monitoring
- ✈️ Flight and logistics network analysis
- 🌡️ Climate and environmental data display
- 🏙️ Urban development trend analysis
- 📊 Geographic statistics dashboard

### 🚀 Advanced Learning:
- **Enterprise Dashboard** - See 08-interactive-3d-dashboard.ipynb
- **Three.js Advanced Rendering** - See 06-threejs-visualization.ipynb
- **React Component Development** - See 03-react-components.ipynb

### 💡 Technical Highlights:
- **Real Geographic Coordinate System** - Precise latitude/longitude mapping
- **High-Performance 3D Rendering** - Smooth display of large data points
- **Rich Visual Effects** - Gradients, lighting, shadows
- **Complete Data Analysis** - Multi-dimensional statistics and insight extraction